In [25]:
from task4.get_data import get_videos_from_folder,get_target_from_csv
import os
import numpy as np
import tensorflow as tf
import helper_functions as hf
from task4.utils import save_solution
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import cross_val_score

In [20]:
dir_path = "input/"
train_folder = os.path.join(dir_path, "train/")
test_folder = os.path.join(dir_path, "test/")

train_target = os.path.join(dir_path, 'train_target.csv')

x_train = get_videos_from_folder(train_folder)
y_train = get_target_from_csv(train_target)
x_test = get_videos_from_folder(test_folder)

In [21]:
my_solution_file = os.path.join("task4/output/", 'solution.csv')

In [28]:
x_train_new = np.nan_to_num(x_train)

In [26]:
clf = GradientBoostingClassifier()

In [29]:
print("========= CrossValidation =========")
results = cross_val_score(clf, x_train_new, y_train, cv=5, n_jobs=1, scoring=hf.scorer())
print("Results: %.4f (%.4f) MSE" % (results.mean(), results.std()))

========= CrossValidation =========


C:\Users\Frankie\PycharmProjects\global_venv_36\lib\site-packages\sklearn\model_selection\_validation.py:542: FutureWarning: From version 0.22, errors during fit will result in a cross validation score of NaN by default. Use error_score='raise' if you want an exception raised or error_score=np.nan to adopt the behavior from version 0.22.
  FutureWarning)


ValueError: setting an array element with a sequence.

In [17]:
save_solution(my_solution_file, dummy_solution)